In [1]:
import argparse
import random
import torch

import numpy as np
import torch.nn.functional as F

from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm

from my_datasets import (
  ParaphraseDetectionDataset,
  ParaphraseDetectionTestDataset,
  load_paraphrase_data
)
from evaluation_reft import model_eval_paraphrase, model_test_paraphrase
from models.gpt2 import GPT2Model

from optimizer import AdamW
import transformers

import pyreft

In [2]:
!export CUDA_VISIBLE_DEVICES=2

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
gpt2 = transformers.AutoModelForCausalLM.from_pretrained('gpt2-large').to(device)
gpt2_tokenizer = transformers.AutoTokenizer.from_pretrained('gpt2-large', device = 'cuda')
gpt2_tokenizer.pad_token = gpt2_tokenizer.unk_token
EOS_TOKEN=gpt2_tokenizer.eos_token

In [10]:
## VANILLA REFT CONFIG


# layers 20, 26, 32, 36 interventions, middle to last
# Each intervention is rank 2-4.

# For sonnet, try first 3/last 3 interventions.
# For classification, try last 3.
reft_config = pyreft.ReftConfig(representations=[{
    "layer": l, "component": "block_output",
    "low_rank_dimension": 4,
    "intervention": pyreft.LoreftIntervention(embed_dim=gpt2.config.hidden_size,
    low_rank_dimension=4)} for l in [19, 24, 29, 35]])
reft_model = pyreft.get_reft_model(gpt2, reft_config)
reft_model.set_device("cuda")
reft_model = reft_model.float()
reft_model.print_trainable_parameters()

Intervention key: layer_19_comp_block_output_unit_pos_nunit_1#0
Intervention key: layer_24_comp_block_output_unit_pos_nunit_1#0
Intervention key: layer_29_comp_block_output_unit_pos_nunit_1#0
Intervention key: layer_35_comp_block_output_unit_pos_nunit_1#0
trainable intervention params: 40,976 || trainable model params: 0
model params: 774,030,080 || trainable%: 0.0052938511123495355


In [11]:
from types import SimpleNamespace

args = SimpleNamespace(
    para_train="data/quora-train.csv",
    para_dev="data/quora-dev.csv",
    para_test="data/quora-test-student.csv",
    para_dev_out="predictions/para-dev-output.csv",
    para_test_out="predictions/para-test-output.csv",
    seed=11711,
    epochs=10,
    use_gpu=True,  # change to True if you want GPU usage
    batch_size=32,
    lr=1e-5,
    model_size="gpt2"
)

In [6]:
para_train_data = load_paraphrase_data(args.para_train)
para_dev_data = load_paraphrase_data(args.para_dev)

para_train_data = ParaphraseDetectionDataset(para_train_data, args, tokenizer = gpt2_tokenizer)
para_dev_data = ParaphraseDetectionDataset(para_dev_data, args, tokenizer = gpt2_tokenizer)

para_train_dataloader = DataLoader(para_train_data, shuffle=True, batch_size=args.batch_size,
                                    collate_fn=para_train_data.collate_fn)
para_dev_dataloader = DataLoader(para_dev_data, shuffle=False, batch_size=args.batch_size,
                                collate_fn=para_dev_data.collate_fn)

Loaded 283003 train examples from data/quora-train.csv
Loaded 40429 train examples from data/quora-dev.csv


In [13]:
inputs = [f'<|user|>:Tell me if these questions are asking the same thing.\nQuestion 1: {p[0]}\nQuestion 2: {p[1]}\nAre these questions asking the same thing?</s>\n<|assistant|>:' for p in para_train_data]
outputs = [('yes' if p[2] == 1 else 'no') for p in para_train_data]
print('DATA LOADED')
positions = 'l3'
data_module = pyreft.make_multiple_position_supervised_data_module(
    gpt2_tokenizer, gpt2, inputs, outputs,
    positions = positions, num_interventions = len(reft_config.representations))

DATA LOADED


In [14]:
training_args = transformers.TrainingArguments(
    num_train_epochs=1, output_dir="./tmp", per_device_train_batch_size=10, 
    learning_rate=5e-5, logging_steps=100,
    lr_scheduler_type=transformers.SchedulerType.LINEAR,
    report_to = [], # disable logging
    warmup_steps=500,
    weight_decay = 0.001
    # L2 regularization
    # weight_decay=0.01
    ) 
trainer = pyreft.ReftTrainerForCausalLM(
    model=reft_model, tokenizer=gpt2_tokenizer, args=training_args, **data_module)
_ = trainer.train()

/tmp/user/23992/ipykernel_2187680/3232451456.py:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `ReftTrainerForCausalLM.__init__`. Use `processing_class` instead.
  trainer = pyreft.ReftTrainerForCausalLM(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss
100,6.917800
200,5.215800
300,2.281600
400,0.414600
500,0.358400
600,0.358600
700,0.338100
800,0.345000
900,0.338400
1000,0.329300


Directory ./tmp/checkpoint-500/intervenable_model already exists and contains files. Skipping save to prevent overwriting existing model.
Directory ./tmp/checkpoint-1000/intervenable_model already exists and contains files. Skipping save to prevent overwriting existing model.
Directory ./tmp/checkpoint-1500/intervenable_model already exists and contains files. Skipping save to prevent overwriting existing model.
Directory ./tmp/checkpoint-2000/intervenable_model already exists and contains files. Skipping save to prevent overwriting existing model.
Directory ./tmp/checkpoint-2500/intervenable_model already exists and contains files. Skipping save to prevent overwriting existing model.
Directory ./tmp/checkpoint-3000/intervenable_model already exists and contains files. Skipping save to prevent overwriting existing model.
Directory ./tmp/checkpoint-3500/intervenable_model already exists and contains files. Skipping save to prevent overwriting existing model.
Directory ./tmp/checkpoint-4

Directory './tmp/checkpoint-28301/intervenable_model' created successfully.


In [19]:
print(inputs[0:5])

['<|user|>:Tell me if these questions are asking the same thing.\nQuestion 1: how do i start learning about artificial intelligence ?\nQuestion 2: how do you learn artificial intelligence ?\nAre these questions asking the same thing?</s>\n<|assistant|>:', '<|user|>:Tell me if these questions are asking the same thing.\nQuestion 1: how can someone learn biochemistry using first principles thinking ?\nQuestion 2: how can someone learn neuroscience using first principles thinking ?\nAre these questions asking the same thing?</s>\n<|assistant|>:', '<|user|>:Tell me if these questions are asking the same thing.\nQuestion 1: how do i view a private broadcast on periscope ?\nQuestion 2: who is alexander khan on periscope ?\nAre these questions asking the same thing?</s>\n<|assistant|>:', '<|user|>:Tell me if these questions are asking the same thing.\nQuestion 1: what are the best companies for android developer in chennai ?\nQuestion 2: is there any best android app development company in hy

In [29]:
print(EOS_TOKEN)

<|endoftext|>


In [15]:
import os
import shutil

save_dir = "./reft_gpt_large_PARAPHRASE_BIGGER_AND_BETTER"
if os.path.exists(save_dir):
    shutil.rmtree(save_dir)  # Remove the existing directory

reft_model.set_device("cpu")  # Move model to CPU before saving
reft_model.save(save_directory=save_dir)

# reft_model.set_device("cpu") # send back to cpu before saving.
# reft_model.save(
#     save_directory="./reft_gpt_large_PARAPHRASE", 
#     overwrite=True,
# )

Directory './reft_gpt_large_PARAPHRASE_BIGGER_AND_BETTER' created successfully.


In [21]:
import torch, transformers, pyreft
device = "cuda"

model_name_or_path = "gpt2-large"
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name_or_path, torch_dtype=torch.bfloat16, device_map=device)

reft_model = pyreft.ReftModel.load(
    "./reft_gpt_large_PARAPHRASE_BIGGER", model
)
reft_model.set_device(device)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
gpt2 = transformers.AutoModelForCausalLM.from_pretrained('gpt2-large').to(device)
gpt2_tokenizer = transformers.AutoTokenizer.from_pretrained('gpt2-large', device = 'cuda')
gpt2_tokenizer.pad_token = gpt2_tokenizer.unk_token
EOS_TOKEN=gpt2_tokenizer.eos_token

Intervention key: layer_34_comp_block_output_unit_pos_nunit_1#0


RuntimeError: Error(s) in loading state_dict for Linear:
	size mismatch for weight: copying a param with shape torch.Size([4, 1280]) from checkpoint, the shape in current model is torch.Size([128, 1280]).
	size mismatch for bias: copying a param with shape torch.Size([4]) from checkpoint, the shape in current model is torch.Size([128]).

In [7]:
prompt = ["<|user|>:Tell me if these questions are asking the same thing.\nQuestion 1: Are you gay?\nQuestion 2: What is the capital of France?\nAre these questions asking the same thing?</s>\n<|assistant|>:"]
prompt = gpt2_tokenizer(prompt, return_tensors="pt").to(device)

full_prompt = gpt2_tokenizer.decode(prompt["input_ids"][0], skip_special_tokens=True)
print(full_prompt)
base_unit_location = prompt["input_ids"].shape[-1] - 1  # last position
_, reft_response = reft_model.generate(
    prompt, unit_locations={"sources->base": (None, [[[base_unit_location]]])},
    intervene_on_prompt=True, max_new_tokens=512, do_sample=True, 
    eos_token_id=gpt2_tokenizer.eos_token_id, early_stopping=True
)
print(gpt2_tokenizer.decode(reft_response[0], skip_special_tokens=True))

/lfs/skampere1/0/ericc27/miniconda3/envs/cs224n/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:677: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<|user|>:Tell me if these questions are asking the same thing.
Question 1: Are you gay?
Question 2: What is the capital of France?
Are these questions asking the same thing?</s>
<|assistant|>:
<|user|>:Tell me if these questions are asking the same thing.
Question 1: Are you gay?
Question 2: What is the capital of France?
Are these questions asking the same thing?</s>
<|assistant|>:no


In [5]:
from evaluation_reft import model_eval_paraphrase_intervenable
model_eval_paraphrase_intervenable(para_dev_dataloader, reft_model, 'cuda', gpt2_tokenizer)

NameError: name 'para_dev_dataloader' is not defined